<a href="https://colab.research.google.com/github/SriRamK345/NLP_dataset/blob/main/Fake_News_Classifier_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [125]:
# https://www.kaggle.com/c/fake-news/data

In [126]:
# ! pip install opendatasets

In [127]:
# import opendatasets as od
# od.download("https://www.kaggle.com/c/fake-news/data")

In [128]:
import pandas as pd

In [129]:
df = pd.read_csv("/content/fake-news/train.csv")

In [130]:
df.head(7)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1


In [131]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [132]:
df.shape

(20800, 5)

In [133]:
df.dropna(inplace=True)

In [134]:
df.reset_index(inplace=True)

In [135]:
X = df.drop("label", axis=1)

In [136]:
y = df["label"]

In [137]:
print(X.shape)
print(y.shape)

(18285, 5)
(18285,)


In [138]:
import tensorflow as tf

In [139]:
print(tf.__version__)

2.17.0


In [140]:
from tensorflow.keras.layers import Embedding
# For creating word embeddings (dense vector representations of words)
from tensorflow.keras.preprocessing.sequence import pad_sequences
# For padding sequences to the same length
from tensorflow.keras.models import Sequential
# For creating a sequential neural network model
from tensorflow.keras.preprocessing.text import one_hot
# For encoding text data into integer sequences using one-hot encoding
from tensorflow.keras.layers import LSTM
# For adding Long Short-Term Memory layers, used in sequence processing (like text)
from tensorflow.keras.layers import Dense
# For adding fully connected layers (dense layers) in the neural network


## Vocabulary size

In [141]:
voc_size = 5000

In [142]:
messages = X.copy()

In [143]:
messages.head(7)

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
6,7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."


In [144]:
messages.isna().sum()

,0
index,0
id,0
title,0
author,0
text,0


In [145]:
# messages.reset_index(inplace=True)

In [146]:
import nltk
import re
import string
from nltk.corpus import stopwords

In [147]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Data Preprocessing

In [148]:
from nltk.stem.porter import PorterStemmer ## stemming
ps = PorterStemmer()

In [149]:
# corpus = []
# for i in range(0, len(messages.title)):
#   try:
#        text = re.sub("[^a-zA-Z]", "" , messages.title[i]) # Remove number and special characters
#   except KeyError:
#      # Handle the case where the index 'i' does not exist
#     print(f"KeyError: Index {i} does not exist in the DataFrame.")
#     continue  # Skip to the next iteration
#   text = text.translate(str.maketrans("", "", string.punctuation)) # Remove punctuation
#   text = text.lower() # Convert test to lowercase
#   text = text.split()
#   text = ([word for word in text if word not in stopwords.words("english")]) # Remove stopwords
#   text = ([ps.stem(word) for word in text]) # stemming
#   text = " ".join(text)
#   corpus.append(text)

In [150]:
corpus = []
for i in range(0, len(messages.title)):
  text = re.sub("[^a-zA-Z]", " " , messages.title[i]) # Remove number and special characters
  text = text.translate(str.maketrans("", "", string.punctuation)) # Remove punctuation
  text = text.lower() # Convert test to lowercase
  text = text.split()
  text = ([word for word in text if word not in stopwords.words("english")]) # Remove stopwords
  text = ([ps.stem(word) for word in text]) # stemming
  text = " ".join(text)
  corpus.append(text)

In [161]:
messages.title[6]

'Benoît Hamon Wins French Socialist Party’s Presidential Nomination - The New York Times'

In [162]:
corpus[6]

'beno hamon win french socialist parti presidenti nomin new york time'

## One hot Representaion

In [153]:
onehot_repr = [one_hot(words, voc_size) for words in corpus] # index of the specfic words
onehot_repr

[[4385, 3067, 151, 132, 980, 2786, 3955, 1569, 2413, 4883],
 [4498, 2895, 3618, 782, 248, 393, 231],
 [3899, 3923, 3999, 3321],
 [2387, 472, 1959, 1739, 2035, 2193],
 [207, 248, 4254, 4666, 1943, 774, 248, 2208, 2574, 4680],
 [244,
  3891,
  2727,
  2055,
  1884,
  3742,
  3017,
  2841,
  3990,
  93,
  1417,
  1292,
  4949,
  4975,
  231],
 [2992, 4429, 3532, 1510, 3136, 4077, 1140, 741, 1737, 4003, 3830],
 [1859, 3983, 3978, 2287, 3062, 4, 3742, 4839, 1737, 4003, 3830],
 [1483, 2203, 1585, 3614, 3215, 4656, 249, 4681, 3742, 1022],
 [4438, 1569, 2937, 2121, 2635, 371, 2744, 4786],
 [2125, 2937, 4141, 2478, 991, 3605, 397, 4258, 1641, 4157, 1258],
 [1739, 2666, 980, 4656, 3742, 3062],
 [4163, 1438, 362, 4518, 4281, 3106, 2587, 1599, 3330],
 [1914, 248, 2082, 2354, 1315, 3129, 92, 1737, 4003, 3830],
 [4386, 2867, 2314, 2511, 711, 1737, 4003, 3830],
 [2197, 2373, 251, 1624, 1745, 4279, 4530, 4272, 1175, 3767],
 [4906, 373, 2895],
 [189, 4522, 4103, 3502, 3742, 3454, 2005, 231],
 [4692, 32

In [154]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [155]:
onehot_repr[0]

[4385, 3067, 151, 132, 980, 2786, 3955, 1569, 2413, 4883]

In [156]:
# Calculate the length of each sentence in the corpus
sentence_lengths = [len(sentence.split()) for sentence in corpus]

# Print the lengths of all sentences
print(sentence_lengths)
print(max(sentence_lengths))
# sentence_lengths.sort(reverse=True)
# print(sentence_lengths)

[10, 7, 4, 6, 10, 15, 11, 11, 10, 8, 11, 6, 9, 10, 8, 10, 3, 8, 9, 6, 12, 7, 12, 8, 12, 7, 8, 8, 12, 12, 9, 5, 4, 8, 9, 8, 4, 9, 9, 10, 9, 10, 12, 3, 10, 6, 11, 5, 9, 8, 9, 5, 11, 9, 12, 9, 10, 10, 8, 8, 4, 5, 7, 10, 11, 8, 8, 9, 8, 12, 9, 4, 9, 7, 11, 1, 10, 12, 7, 10, 10, 11, 16, 6, 8, 7, 10, 10, 7, 9, 12, 8, 5, 7, 11, 9, 11, 13, 10, 13, 11, 12, 12, 12, 9, 9, 11, 9, 11, 10, 7, 10, 9, 10, 9, 8, 5, 8, 11, 10, 4, 9, 9, 10, 11, 9, 9, 9, 6, 8, 11, 10, 11, 7, 10, 11, 2, 11, 10, 2, 10, 5, 9, 4, 7, 13, 10, 7, 8, 10, 7, 6, 9, 17, 11, 6, 10, 10, 7, 11, 13, 7, 7, 10, 8, 17, 11, 7, 7, 7, 4, 9, 11, 12, 8, 7, 13, 15, 8, 4, 9, 7, 9, 8, 7, 5, 9, 11, 10, 12, 11, 6, 14, 8, 10, 7, 9, 8, 10, 11, 14, 11, 9, 11, 5, 9, 8, 15, 10, 9, 10, 8, 9, 2, 12, 10, 10, 11, 10, 8, 1, 10, 9, 5, 13, 8, 11, 11, 9, 7, 6, 12, 8, 12, 13, 9, 7, 8, 7, 3, 9, 11, 8, 10, 7, 7, 14, 4, 10, 7, 9, 10, 9, 10, 6, 8, 11, 11, 6, 8, 6, 12, 8, 8, 10, 9, 6, 10, 8, 8, 9, 7, 11, 9, 6, 11, 7, 8, 9, 8, 3, 10, 12, 13, 8, 9, 16, 9, 6, 3, 7, 13, 7

## Embedding layer Representation

In [157]:
sent_length = 25
embedded_docs = pad_sequences(onehot_repr, padding="pre", maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1569 2413 4883]
 [   0    0    0 ...  248  393  231]
 [   0    0    0 ... 3923 3999 3321]
 ...
 [   0    0    0 ... 1737 4003 3830]
 [   0    0    0 ... 3472 1438 2521]
 [   0    0    0 ... 3100  156 4775]]


In [158]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0, 4498, 2895, 3618,  782,
        248,  393,  231], dtype=int32)

## Model Creation

In [159]:
embedding_vector_features = 40 # feature representation
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100)) # neuron
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [160]:
len(embedded_docs), y.shape

(18285, (18285,))

## Train test Split

In [164]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [165]:
X_final.shape, y_final.shape

((18285, 25), (18285,))

In [166]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Model training

In [167]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 14s 60ms/step - accuracy: 0.7819 - loss: 0.4256 - val_accuracy: 0.9152 - val_loss: 0.1927
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.9459 - loss: 0.1380 - val_accuracy: 0.9175 - val_loss: 0.1957
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 55ms/step - accuracy: 0.9683 - loss: 0.0906 - val_accuracy: 0.9162 - val_loss: 0.2190
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 19s 49ms/step - accuracy: 0.9755 - loss: 0.0714 - val_accuracy: 0.9145 - val_loss: 0.2527
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 55ms/step - accuracy: 0.9849 - loss: 0.0490 - val_accuracy: 0.9150 - val_loss: 0.2706
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 20s 54ms/step - accuracy: 0.9906 - loss: 0.0354 - val_accuracy: 0.9118 - val_loss: 0.3208
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 19s 45ms/step - accuracy: 0.9931 - loss: 0.0234 - val_accuracy: 0.9072 - val_loss: 0.3573
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 56ms/step - accuracy: 0.9978 - loss: 0.0138 - v

## Adding Dropout

In [ ]:
# from tensorflow.keras.layers import Dropout
# ## Creating model
# embedding_vector_features=40
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(Dropout(0.3)) # To reduce overfitting
# model.add(LSTM(100))
# model.add(Dropout(0.3))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

## Performance Metrics And Accuracy

In [169]:
y_pred = model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step


In [170]:
y_pred = np.where(y_pred > 0.5, 1, 0)

In [171]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [172]:
confusion_matrix(y_test, y_pred)

array([[3146,  273],
       [ 314, 2302]])

In [173]:
print(accuracy_score(y_test, y_pred))

0.9027340513670257


In [175]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.92      0.91      3419
           1       0.89      0.88      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

